In [1]:
import pyforest

In [2]:
data_path = '../datasets/original_datasets/'

In [3]:
test = pd.read_csv(data_path+'NEK2_1_uM_min_50_pct_binding.csv')
test

<IPython.core.display.Javascript object>

,target_name,compound_id,base_rdkit_smiles,pct_binding,active
0,NEK2,kdb_10,CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)c...,3.225806,0
1,NEK2,kdb_100,C[C@@H](Oc1cc(C(=O)Nc2ccc(C(=O)N3CCN(C)CC3)cc2...,3.225806,0
2,NEK2,kdb_1003,CC1(O)CC(c2nc(-c3ccc4ccc(-c5ccccc5)nc4c3)c3c(N...,5.000000,0
3,NEK2,kdb_101,COCC(=O)NC/C=C/c1ccc2ncnc(Nc3ccc(Oc4ccc(C)nc4)...,6.158358,0
4,NEK2,kdb_102,COC(=O)c1ccc2c(c1)NC(=O)/C2=C(\Nc1ccc(N(C)C(=O...,6.158358,0
...,...,...,...,...,...
1403,NEK2,kdb_976,Oc1c2ccccc2c2c3c(cccc13)N=N2,9.090909,0
1404,NEK2,kdb_98,Cc1nc(Nc2n[nH]c3c2CN(C(=O)N[C@H](CN(C)C)c2cccc...,3.225806,0
1405,NEK2,kdb_980,Cc1cnc(-c2cnc(NCCNc3ccc(C#N)cn3)nc2-c2ccc(Cl)c...,85.000000,1
1406,NEK2,kdb_986,Nc1n[nH]c2nnc(-c3ccccc3)c(-c3ccccc3)c12,0.000000,0


In [4]:
def check_labels(data_path, filename): 

  
    df = pd.read_csv(data_path+filename)

    pct_col = [col for col in df.columns if col.startswith('pct_')] 
    len(pct_col) == 1 
    pct_col = pct_col[0]
    invalid_labels = df[df['active']!= (df[pct_col]>=50).astype(int)]
    if invalid_labels.empty: 
        # print(f'correct label assignment based on {pct_col} column')
        pass
    else: 
        print(f'Error on label assignement.')
        print(invalid_labels)
    # check for duplicates here 
    duplicates = df[df.duplicated()]
    print(f'{df.shape}')
    #duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
   
    if not duplicates.empty:
        print('Duplicate rows found:')
        print(duplicates)
        df_cleaned = df.drop_duplicates()
        print(f'df size original: {df.shape}')
        print(f'remove duplicates. {df_cleaned.shape}')
check_labels(data_path,'NEK2_1_uM_min_50_pct_inhibition.csv')

<IPython.core.display.Javascript object>

(2032, 5)


In [5]:
#create a dataset with folds

# must specify filename because the 50pct binding and inhibition files differ
# to-do: discuss how to do this --> this will create a file that is just this but with column specifying split
#       do we want to already integrate a split_uuid like AMPL? 
#       additionally, we still have to use the splits to create the "scaled df" 
def remove_duplicates(datapath, filename):
    df = pd.read_csv(f'{datapath}{filename}')
    duplicates = df.duplicated(subset=['base_rdkit_smiles'])
    if duplicates.any():
        print('Duplicate rows found:')
        print(df[duplicates])
        df_cleaned = df.drop_duplicates(subset=['base_rdkit_smiles'])
        print(f'df size original: {df.shape}')
        print(f'Removed duplicates. New df size: {df_cleaned.shape}')
    else: 
        df_cleaned = df
    return df_cleaned
    
def create_folds(datapath, filename, k_splits): 
    """filename: NEK#_1_uM_min_50_pct_(binding or inhibition).csv"""
    df = remove_duplicates(datapath, filename)
    # is this extra/too much and should we just assume what labels coincide with majority and minority? 
    # determine majority/minority class 
    #       AMPL takes in 'response column' 
    class_labels = df['active'].value_counts() 
    print(class_labels)
    if len(class_labels)>1: 
        majority_class_label =class_labels.idxmax() 
        majority_num = class_labels.max() 
        minority_class_label = class_labels.idxmin()
        minority_count = class_labels.min()
    df_majority = df[df['active']==majority_class_label]
    df_minority=df[df['active']==minority_class_label]
    # copy to avoid warnings 
    df_majority = df_majority.copy()
    df_minority = df_minority.copy()
    kf = KFold(k_splits,shuffle=True, random_state=42)
    # majority 
    for i, (_, v_ind) in enumerate(kf.split(df_majority)):
        df_majority.loc[df_majority.index[v_ind], 'fold'] = f"fold{i+1}"
    # minority 
    for i, (_, v_ind) in enumerate(kf.split(df_minority)):
        df_minority.loc[df_minority.index[v_ind], 'fold'] = f"fold{i+1}"
    all_fold_df = pd.concat([df_majority,df_minority])
    
    print(all_fold_df.shape)
    print(all_fold_df.active.value_counts())
    ## actually, this might be perfect jsut to save as "split df" because then 
    # in the next step of creating a dataset, you can choose what fold you want to be the test set 
    return all_fold_df
        
    

In [6]:
call in create_folds labeled with nek_folded 
    then call train_and__ to get full 80% train and do not touch 20% 
        then get 80% train and call create_folds
            then send to a seperate directory call K-fold 
    
in the the K-fold directory use the data that is folded called train___ 
    use to make train and test with each fold

SyntaxError: invalid syntax (2774517515.py, line 1)

In [16]:
def train_and__(data_path, fold_df, test_fold):
   """function to label 'train' or 'test' in the 'subset' column
   to be used to create train/test OR train/val
   fold_df: dataframe with column 'fold'
   test_fold (str): fold to make the test set (the remaining folds will be train)
   """
   df = pd.read_csv(data_path+fold_df) 
   df['subset'] = df['fold'].apply(lambda x: 'test' if x == test_fold else 'train')

   return df[['NEK','compound_id', 'active', 'base_rdkit_smiles', 'subset']]



In [12]:
def get_bind_inhib(df_name):
    match = re.search(r'_([^_]+)\.csv$',df_name)
    return match.group(1) if match else None
    # returns 'binding' or 'inhibition' 
og_dfs = [file for file in os.listdir(data_path) if file.startswith('NEK')]

for df in og_dfs: 
    folded_df = create_folds(data_path, df,5)
    bind_inhib = get_bind_inhib(df)
    nek_name = f'NEK{df[3]}_{bind_inhib}'
    print(nek_name)
    folded_df['NEK']=nek_name 
    folded_df.to_csv(f'../datasets/initial_fold/{nek_name}_initial_fold.csv',index=False)

    # folded_df.to_csv('intial_fold')

# def label_train_validation(data_path, train80df, fold)
#     return train df with folds

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

active
0    351
1     42
Name: count, dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(1408, 6)
active
0    1351
1      57
Name: count, dtype: int64


<IPython.core.display.Javascript object>

NEK2_binding


In [25]:
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding','NEK9_binding','NEK9_inhibition']

for nek in neks:
    fold_df_name = nek+"_initial_fold.csv"
    print(fold_df_name)
    labeled_df = train_and__("/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/initial_fold/" , fold_df_name, "fold1")
    labeled_df.to_csv(f'/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/{nek}_80_20_df.csv',index=False)
#labeled_df
    



NEK2_binding_initial_fold.csv


<IPython.core.display.Javascript object>

NEK2_inhibition_initial_fold.csv


<IPython.core.display.Javascript object>

NEK3_binding_initial_fold.csv


...

<IPython.core.display.Javascript object>

NEK9_binding_initial_fold.csv


<IPython.core.display.Javascript object>

NEK9_inhibition_initial_fold.csv


<IPython.core.display.Javascript object>